In [1]:
%load_ext jupyter_black
%reload_ext jupyter_black

# Aggiornamento di un dataset con clear ml
Supponiamo che vogliamo fare un aggiornamento di un dataset con clear ml. In particolare, supponiamo di voler aggiornare un dataset con delle immagini ridimensionate perchè il dataset presenta immagini di dimensioni diverse e anche molto grandi per un modello di  Deep Learning.  In questo potremo usare la libreria clearml per fare l'aggiornamento del dataset per noi e per il resto del team, in modo che tutti possano avere accesso al dataset aggiornato e non devono stare a fare un preprocessing delle immagini già fatte.

In [2]:
%env CLEARML_WEB_HOST=http://192.168.1.52:8080
%env CLEARML_API_HOST=http://192.168.1.52:8008
%env CLEARML_FILES_HOST=http://192.168.1.52:8081
%env CLEARML_API_ACCESS_KEY=PV827LMYHMLGBY8OP8YK
%env CLEARML_API_SECRET_KEY=XziInTQagGSCtZ8JA14JYpyCrc1CEYQKmf0HLSINhDPtlgS6kt

env: CLEARML_WEB_HOST=http://192.168.1.52:8080
env: CLEARML_API_HOST=http://192.168.1.52:8008
env: CLEARML_FILES_HOST=http://192.168.1.52:8081
env: CLEARML_API_ACCESS_KEY=PV827LMYHMLGBY8OP8YK
env: CLEARML_API_SECRET_KEY=XziInTQagGSCtZ8JA14JYpyCrc1CEYQKmf0HLSINhDPtlgS6kt


In [2]:
from pathlib import Path
from clearml import Task, Dataset, TaskTypes
import pandas as pd
import numpy as np
import shutil

# Creazione di un task
Anche qui creiamo un task per poter la creazione di questo preprocessing delle immagini.

In [3]:
task = Task.init(project_name="fruit_classification_with_vit", task_name="pre_process_data", task_type= TaskTypes.data_processing)

ClearML Task: overwriting (reusing) task id=041f5bdce89b4a2b9efde7d1414e2adb
ClearML results page: http://192.168.1.52:8080/projects/13214ae35e3d4b44801736a36c54a6a5/experiments/041f5bdce89b4a2b9efde7d1414e2adb/output/log
2024-03-07 16:51:20,720 - clearml.Task - INFO - Storing jupyter notebook directly as code


In [4]:
##download data from clearML
dataset = Dataset.get(dataset_id='cfa7b0b9054e4d6baed1260e12eb7cbd')

2024-03-07 16:51:22,432 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.


In [5]:
dataset.get_mutable_local_copy('../data/fruit_dataset')

'/home/matteo/PycharmProjects/clearML/notebooks/../data/fruit_dataset'

In [6]:
path_to_data = Path('..',"data", 'fruit_dataset')
choices = ['train_set', 'val_set', 'test_set']
probabilities = [0.7, 0.15, 0.15] # possono diventare dei paramatri da selezionare con argparse
seed = 42
np.random.seed(42) #posso impostare argparser

In [7]:
resources = []
folders = []
filenames = []
sets = []

for folder in path_to_data.iterdir():
    if folder.is_dir():
        #folders.append(folder.name)
        for resource in folder.iterdir():
            if resource.is_file():
                resources.append(resource.as_posix())
                filenames.append(resource.name)
                folders.append(folder.name)
        #num_images = len(list(folder.glob("*.jpg")))
        #result = np.random.choice(choices, size=num_images, p=probabilities)
        train_set = ['train_set']*28
        val_set = ['val_set']*6
        test_set = ['test_set']*6
        result = train_set + val_set + test_set
        result = np.random.permutation(result)
        sets.extend(result)
    

data = [resources, filenames, folders, sets]
df = pd.DataFrame(data, index=["resources", "filenames" ,"folders", "sets"]).T
df

,resources,filenames,folders,sets
0,../data/fruit_dataset/grapes fruit/Image_25.jpg,Image_25.jpg,grapes fruit,train_set
1,../data/fruit_dataset/grapes fruit/Image_10.jpg,Image_10.jpg,grapes fruit,train_set
2,../data/fruit_dataset/grapes fruit/Image_24.jpg,Image_24.jpg,grapes fruit,train_set
3,../data/fruit_dataset/grapes fruit/Image_33.jpg,Image_33.jpg,grapes fruit,train_set
4,../data/fruit_dataset/grapes fruit/Image_20.jpg,Image_20.jpg,grapes fruit,train_set
...,...,...,...,...
355,../data/fruit_dataset/cherry fruit/Image_13.jpg,Image_13.jpg,cherry fruit,val_set
356,../data/fruit_dataset/cherry fruit/Image_7.jpg,Image_7.jpg,cherry fruit,train_set
357,../data/fruit_dataset/cherry fruit/Image_39.jpg,Image_39.jpg,cherry fruit,val_set
358,../data/fruit_dataset/cherry fruit/Image_37.jpg,Image_37.jpg,cherry fruit,test_set


In [8]:
def new_filename(row):
    filename = row['filenames']
    name, ext = filename.split(".")
    new_filename = f"{name}-{row['folders']}.{ext}".lower()
    return new_filename

In [9]:
df['folders'] = df['folders'].str.replace(" ", "_")
df['new_filenames'] = df.apply(new_filename, axis=1)
df

,resources,filenames,folders,sets,new_filenames
0,../data/fruit_dataset/grapes fruit/Image_25.jpg,Image_25.jpg,grapes_fruit,train_set,image_25-grapes_fruit.jpg
1,../data/fruit_dataset/grapes fruit/Image_10.jpg,Image_10.jpg,grapes_fruit,train_set,image_10-grapes_fruit.jpg
2,../data/fruit_dataset/grapes fruit/Image_24.jpg,Image_24.jpg,grapes_fruit,train_set,image_24-grapes_fruit.jpg
3,../data/fruit_dataset/grapes fruit/Image_33.jpg,Image_33.jpg,grapes_fruit,train_set,image_33-grapes_fruit.jpg
4,../data/fruit_dataset/grapes fruit/Image_20.jpg,Image_20.jpg,grapes_fruit,train_set,image_20-grapes_fruit.jpg
...,...,...,...,...,...
355,../data/fruit_dataset/cherry fruit/Image_13.jpg,Image_13.jpg,cherry_fruit,val_set,image_13-cherry_fruit.jpg
356,../data/fruit_dataset/cherry fruit/Image_7.jpg,Image_7.jpg,cherry_fruit,train_set,image_7-cherry_fruit.jpg
357,../data/fruit_dataset/cherry fruit/Image_39.jpg,Image_39.jpg,cherry_fruit,val_set,image_39-cherry_fruit.jpg
358,../data/fruit_dataset/cherry fruit/Image_37.jpg,Image_37.jpg,cherry_fruit,test_set,image_37-cherry_fruit.jpg


In [10]:
path_to_store_data = Path('..', "data", "fruit_dataset_preprocessed")
path_to_store_data.mkdir(exist_ok=True, parents=True)
path_to_store_data.joinpath("train_set").mkdir(exist_ok=True)
path_to_store_data.joinpath("val_set").mkdir(exist_ok=True)
path_to_store_data.joinpath("test_set").mkdir(exist_ok=True)
for idx, row in df.iterrows():
    old_path = row['resources']
    new_path = path_to_store_data.joinpath(row['sets'], row['new_filenames'])
    shutil.copy(old_path, new_path)

In [11]:
training_set = Dataset.create(dataset_name="fruit_dataset", dataset_project="fruit_classification_with_vit", parent_datasets=[dataset.id], dataset_tags= ['training_set'])
validation_set = Dataset.create(dataset_name="fruit_dataset", dataset_project="fruit_classification_with_vit", parent_datasets=[dataset.id], dataset_tags= ['validation_set'])
test_set = Dataset.create(dataset_name="fruit_dataset", dataset_project="fruit_classification_with_vit", parent_datasets=[dataset.id],
                          dataset_tags= ['test_set'])

2024-03-07 16:51:37,300 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
2024-03-07 16:51:39,595 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
ClearML results page: http://192.168.1.52:8080/projects/8e6a5bbbfb3f4d27b42d265f9c52a990/experiments/475b8e9649a647978db2ebb5f8dcd300/output/log
ClearML dataset page: http://192.168.1.52:8080/datasets/simple/8e6a5bbbfb3f4d27b42d265f9c52a990/experiments/475b8e9649a647978db2ebb5f8dcd300
2024-03-07 16:51:40,500 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
2024-03-07 16:51:41,367 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
ClearML results page: http://192.168.1.52:8080/projects/8e6a5bbbfb3f4d27b42d265f9c52a990/experiments/746d0

In [12]:
training_set.sync_folder(local_path=path_to_store_data.joinpath('train_set').as_posix(), verbose=True)
validation_set.sync_folder(local_path=path_to_store_data.joinpath('val_set').as_posix(), verbose=True)
test_set.sync_folder(local_path=path_to_store_data.joinpath('test_set').as_posix(), verbose=True)

Remove grapes fruit/Image_30.jpeg
Remove grapes fruit/Image_12.jpg
Remove apple fruit/Image_23.png
Remove mango fruit/Image_11.jpg
Remove grapes fruit/Image_18.jpg
Remove chickoo fruit/Image_24.jpg
Remove mango fruit/Image_20.jpg
Remove orange fruit/Image_1.jpg
Remove banana fruit/Image_10.png
Remove mango fruit/Image_15.jpg
Remove cherry fruit/Image_13.jpg
Remove orange fruit/Image_9.jpg
Remove banana fruit/Image_18.jpg
Remove orange fruit/Image_39.JPG
Remove grapes fruit/Image_22.jpg
Remove orange fruit/Image_21.jpg
Remove kiwi fruit/Image_19.jpg
Remove cherry fruit/Image_38.jpg
Remove banana fruit/Image_19.jpg
Remove apple fruit/Image_1.jpg
Remove banana fruit/Image_22.jpg
Remove apple fruit/Image_9.jpg
Remove mango fruit/Image_24.jpg
Remove cherry fruit/Image_25.jpg
Remove apple fruit/Image_21.jpg
Remove apple fruit/Image_7.jpg
Remove orange fruit/Image_14.jpg
Remove chickoo fruit/Image_39.jpg
Remove strawberry fruit/Image_13.jpg
Remove cherry fruit/Image_8.jpg
Remove orange fruit/

100%|██████████| 252/252 [00:00<00:00, 4047.98it/s]

Hash generation completed
Add image_33-kiwi_fruit.jpg
Add image_19-strawberry_fruit.jpg
Add image_33-cherry_fruit.jpg
Add image_30-apple_fruit.jpg
Add image_8-cherry_fruit.jpg
Add image_28-strawberry_fruit.jpg
Add image_21-grapes_fruit.jpg
Add image_27-mango_fruit.jpeg
Add image_9-orange_fruit.jpg
Add image_3-chickoo_fruit.png
Add image_26-cherry_fruit.jpg
Add image_22-banana_fruit.jpg
Add image_6-orange_fruit.jpg
Add image_14-grapes_fruit.jpg
Add image_11-chickoo_fruit.jpg
Add image_33-banana_fruit.jpg
Add image_30-cherry_fruit.jpg
Add image_4-grapes_fruit.jpg
Add image_38-strawberry_fruit.jpg
Add image_28-chickoo_fruit.png
Add image_3-banana_fruit.jpeg
Add image_37-chickoo_fruit.jpg
Add image_4-kiwi_fruit.jpg
Add image_35-banana_fruit.jpg
Add image_39-kiwi_fruit.jpg
Add image_5-banana_fruit.jpg
Add image_27-kiwi_fruit.jpg
Add image_34-banana_fruit.jpg
Add image_12-chickoo_fruit.jpg
Add image_10-orange_fruit.jpg
Add image_18-apple_fruit.jpg
Add image_2-cherry_fruit.png
Add image_3-app

Add image_27-apple_fruit.jpg
Add image_27-grapes_fruit.jpg
Add image_4-chickoo_fruit.jpg
Add image_36-mango_fruit.jpg
Add image_9-kiwi_fruit.jpg
Add image_7-orange_fruit.jpg
Add image_36-grapes_fruit.jpeg
Add image_1-chickoo_fruit.jpg
Add image_22-mango_fruit.jpg
Add image_22-kiwi_fruit.jpg
Add image_38-grapes_fruit.jpg
Add image_5-apple_fruit.jpg
Add image_2-chickoo_fruit.jpg
Add image_22-strawberry_fruit.jpg
Add image_21-kiwi_fruit.png
Add image_16-apple_fruit.jpg
Add image_32-mango_fruit.jpg
Add image_8-mango_fruit.jpg
Add image_28-cherry_fruit.jpg
Add image_39-mango_fruit.jpg
Add image_18-banana_fruit.jpg
Add image_19-chickoo_fruit.jpg
Add image_14-banana_fruit.jpg
Add image_21-strawberry_fruit.jpg
Add image_29-orange_fruit.jpg
Add image_34-strawberry_fruit.jpeg
Add image_30-banana_fruit.jpg
Add image_13-orange_fruit.jpg
Add image_12-grapes_fruit.jpg
Add image_38-mango_fruit.jpg
Add image_40-cherry_fruit.jpg
Add image_2-apple_fruit.jpg
Add image_21-chickoo_fruit.jpg
Add image_32-ch

100%|██████████| 54/54 [00:00<00:00, 1097.91it/s]


Hash generation completed
Add image_3-orange_fruit.jpg
Add image_24-apple_fruit.jpg
Add image_11-mango_fruit.jpg
Add image_26-chickoo_fruit.jpg
Add image_19-kiwi_fruit.jpg
Add image_35-mango_fruit.jpg
Add image_29-chickoo_fruit.jpg
Add image_36-banana_fruit.jpg
Add image_18-chickoo_fruit.jpg
Add image_5-chickoo_fruit.jpg
Add image_11-orange_fruit.jpg
Add image_39-cherry_fruit.jpg
Add image_23-grapes_fruit.jpg
Add image_7-banana_fruit.jpg
Add image_4-orange_fruit.jpg
Add image_9-cherry_fruit.jpg
Add image_2-orange_fruit.jpg
Add image_15-orange_fruit.jpg
Add image_26-strawberry_fruit.jpg
Add image_14-kiwi_fruit.jpg
Add image_29-kiwi_fruit.jpg
Add image_11-apple_fruit.jpg
Add image_31-apple_fruit.png
Add image_1-cherry_fruit.jpg
Add image_12-orange_fruit.jpg
Add image_15-mango_fruit.jpg
Add image_15-chickoo_fruit.png
Add image_26-grapes_fruit.jpg
Add image_29-strawberry_fruit.jpg
Add image_15-cherry_fruit.jpg
Add image_36-cherry_fruit.jpg
Add image_17-banana_fruit.jpg
Add image_39-grapes_

100%|██████████| 54/54 [00:00<00:00, 2260.38it/s]


Hash generation completed
Add image_32-grapes_fruit.jpg
Add image_2-banana_fruit.jpg
Add image_11-banana_fruit.jpg
Add image_6-strawberry_fruit.jpg
Add image_1-banana_fruit.jpg
Add image_6-banana_fruit.jpg
Add image_37-grapes_fruit.jpg
Add image_8-chickoo_fruit.jpg
Add image_24-orange_fruit.jpg
Add image_35-strawberry_fruit.jpg
Add image_37-apple_fruit.jpg
Add image_7-mango_fruit.jpg
Add image_22-grapes_fruit.jpg
Add image_34-kiwi_fruit.jpg
Add image_32-banana_fruit.jpg
Add image_32-chickoo_fruit.jpg
Add image_7-chickoo_fruit.jpg
Add image_21-cherry_fruit.jpg
Add image_3-strawberry_fruit.jpg
Add image_28-apple_fruit.png
Add image_1-mango_fruit.jpg
Add image_28-grapes_fruit.jpg
Add image_37-cherry_fruit.jpg
Add image_40-apple_fruit.jpeg
Add image_25-apple_fruit.jpg
Add image_39-strawberry_fruit.jpg
Add image_17-grapes_fruit.jpg
Add image_5-orange_fruit.jpg
Add image_11-cherry_fruit.jpeg
Add image_31-kiwi_fruit.jpg
Add image_4-mango_fruit.jpg
Add image_8-grapes_fruit.jpg
Add image_4-bana

(360, 54, 0)

In [13]:
validation_set.finalize(verbose=True, auto_upload=True)

Pending uploads, starting dataset upload to http://192.168.1.52:8081
Uploading dataset changes (54 files compressed to 45.49 MiB) to http://192.168.1.52:8081
2024-03-07 16:51:45,462 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
File compression and upload completed: total size 45.49 MiB, 1 chunk(s) stored (average size 45.49 MiB)
2024-03-07 16:51:50,013 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
2024-03-07 16:51:50,928 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
Updating statistics and genealogy
2024-03-07 16:51:51,620 - clearml - INFO - Dataset.get() did not specify alias. Dataset information will not be automatically logged in ClearML Server.
2024-03-07 16:51:51,987 - clearml - INFO - Dataset.get() did not specify alias. Dataset informat

True

In [14]:
test_set.finalize(verbose=True, auto_upload=True)

Pending uploads, starting dataset upload to http://192.168.1.52:8081
Uploading dataset changes (54 files compressed to 33.34 MiB) to http://192.168.1.52:8081
File compression and upload completed: total size 33.34 MiB, 1 chunk(s) stored (average size 33.34 MiB)
Updating statistics and genealogy


True

In [15]:
task.close()